# building cross-compiler GNU toolchain

Cross-compiler build is going using GNU [make](make.ipynb) tool, which is the minimal and still powerful automation tool for build control.

## target system configuration

configuration variables can be overriden in a command line:
```
cd msp430 ; make APP=cnc HW=pc104 TARGET=i686 binutils
```

### application

In [372]:
%%writefile msp430/Makefile

APP    ?= blink

Overwriting msp430/Makefile


### hardware device

Hardware device you are targetting, must be used for lot of configuration: GPIO ports, peripherial pingout,..

* `HW`
  * [msp430g2]()
  * [cm0pill]()
  * [cm1pill]()

In [373]:
%%writefile -a msp430/Makefile

HW     ?= msp430g2

Appending to msp430/Makefile


### target MCU/CPU/OS triplet

* `BUILD`
  - system name of your workstation OS in form of **system triplet**
    - `x86_64-pc-linux-gnu`
* `HOST`
  - triplet of OS will run on an emLinux embedded device ([Raspberry](), [PC104](),..)
    * for MCU targets we'll drop it
* `TARGET`
  - triplet of MCU target
    * [msp430](msp430.ipynb)
    * i386

In [374]:
%%writefile -a msp430/Makefile

TARGET ?= msp430

Appending to msp430/Makefile


## directory structure

This is set of directories will be created 

* `CWD` in the same dir as `Makefile` itself or directory selected by `make -C` option
* `GZ` downloaded archives
* `TMP` temp build directory
* `SRC` source code for every tool will be built
* `GNU` all compilers and tools will be pushed locally into this directory
  - such a scheme will not affect to whole `BUILD`/`HOST` system, all toolds will reside in a user `HOME`

In [375]:
%%writefile -a msp430/Makefile

CWD = $(CURDIR)
GZ  = $(CWD)/gz
TMP = $(CWD)/tmp
SRC = $(TMP)/src
GNU = $(CWD)/gnu_$(TARGET)

dirs:
	mkdir -p $(GZ) $(TMP) $(SRC) $(GNU)

Appending to msp430/Makefile


In [376]:
%%sh
cd msp430 ; make dirs ; ls -la ; ls -la tmp

mkdir -p /home/dpon/jupy/msp430/gz /home/dpon/jupy/msp430/tmp /home/dpon/jupy/msp430/tmp/src /home/dpon/jupy/msp430/gnu_msp430
итого 44
drwxr-xr-x  8 dpon dpon 4096 май 29 16:41 .
drwxr-xr-x 11 dpon dpon 4096 май 29 17:37 ..
-rw-r--r--  1 dpon dpon  724 май 29 15:04 cross.ipynb
drwxr-xr-x  2 dpon dpon 4096 май 29 16:40 cross_msp430
-rw-r--r--  1 dpon dpon   21 май 29 16:41 .gitignore
drwxr-xr-x  5 dpon dpon 4096 май 29 17:36 gnu_msp430
drwxr-xr-x  2 dpon dpon 4096 май 29 16:29 gz
drwxr-xr-x  2 dpon dpon 4096 май 29 14:41 .ipynb_checkpoints
-rw-r--r--  1 dpon dpon  193 май 29 17:37 Makefile
drwxr-xr-x  2 dpon dpon 4096 май 29 16:39 msp430
drwxr-xr-x  4 dpon dpon 4096 май 29 17:35 tmp
итого 16
drwxr-xr-x  4 dpon dpon 4096 май 29 17:35 .
drwxr-xr-x  8 dpon dpon 4096 май 29 16:41 ..
drwxr-xr-x 12 dpon dpon 4096 май 29 17:35 binutils-2.32
drwxr-xr-x  3 dpon dpon 4096 май 29 17:09 src


## cross-compiler components

## versions

In [377]:
%%writefile -a msp430/Makefile

BINUTILS_VER = 2.32

Appending to msp430/Makefile


In [378]:
%%writefile -a msp430/Makefile

BINUTILS = binutils-$(BINUTILS_VER)

Appending to msp430/Makefile


## predownload all archives

In [379]:
%%writefile -a msp430/Makefile

BINUTILS_GZ = $(BINUTILS).tar.xz

Appending to msp430/Makefile


In [380]:
%%writefile -a msp430/Makefile

WGET = wget -c -P $(GZ) -q

gz: $(GZ)/$(BINUTILS_GZ)

Appending to msp430/Makefile


In [381]:
%%sh
cd msp430 ; make gz ; ls -la gz

make: Цель «gz» не требует выполнения команд.
итого 20296
drwxr-xr-x 2 dpon dpon     4096 май 29 16:29 .
drwxr-xr-x 8 dpon dpon     4096 май 29 16:41 ..
-rw-r--r-- 1 dpon dpon 20774880 фев  2 20:38 binutils-2.32.tar.xz


### archive unpack rules

In [382]:
%%writefile -a msp430/Makefile

$(SRC)/%/README: $(GZ)/%.tar.xz
	cd $(SRC) ; xzcat $< | tar -x && touch $@

Appending to msp430/Makefile


### configure scripts run

In [383]:
%%writefile -a msp430/Makefile

XPATH = PATH=$(GNU)/bin:$(PATH)

CFG = configure --disable-nls --prefix=$(GNU)

MAKE = $(XPATH) make

CPU_CORES = $(shell grep processor /proc/cpuinfo|wc -l)

MAKEJ = $(MAKE) -j$(CPU_CORES)

xp:
	echo $(MAKEJ)

Appending to msp430/Makefile


In [384]:
%%sh
cd msp430 ; make xp

echo PATH=/home/dpon/jupy/msp430/gnu_msp430/bin:/home/dpon/jupy/bin:/home/dpon/jupy/bin:/home/dpon/.cargo/bin:/home/dpon/bin:/home/dpon/qm/bin:/home/dpon/qm/qtools/bin:/home/dpon/.node/bin:/home/dpon/go/bin:/usr/local/go/bin:/home/dpon/Android/SDK/tools:/home/dpon/Android/SDK/platform-tools:/home/dpon/jdk1.8.0_162/bin:/home/dpon/eclipse:/home/dpon/bin:/home/dpon/qm/bin:/home/dpon/qm/qtools/bin:/home/dpon/.node/bin:/home/dpon/go/bin:/usr/local/go/bin:/home/dpon/Android/SDK/tools:/home/dpon/Android/SDK/platform-tools:/home/dpon/jdk1.8.0_162/bin:/home/dpon/eclipse:/usr/local/bin:/usr/bin:/bin:/usr/games make -j4
PATH=/home/dpon/jupy/msp430/gnu_msp430/bin:/home/dpon/jupy/bin:/home/dpon/jupy/bin:/home/dpon/.cargo/bin:/home/dpon/bin:/home/dpon/qm/bin:/home/dpon/qm/qtools/bin:/home/dpon/.node/bin:/home/dpon/go/bin:/usr/local/go/bin:/home/dpon/Android/SDK/tools:/home/dpon/Android/SDK/platform-tools:/home/dpon/jdk1.8.0_162/bin:/home/dpon/eclipse:/home/dpon/bin:/home/dpon/qm/bin:/home/dpon/qm/qt

### [binutils](binutils.ipynb) : assembler, linker, object file tools

In [385]:
%%writefile -a msp430/Makefile

binutils: $(GNU)/bin/$(TARGET)-ld

CFG_BINUTILS += --target=$(TARGET)

$(GNU)/bin/$(TARGET)-ld: $(GZ)/$(BINUTILS_GZ) $(SRC)/$(BINUTILS)/README
	rm -rf $(TMP)/$(BINUTILS) ; mkdir $(TMP)/$(BINUTILS) ; cd $(TMP)/$(BINUTILS) ;\
	$(XPATH) $(SRC)/$(BINUTILS)/$(CFG) $(CFG_BINUTILS) && $(MAKEJ) && $(MAKE) install-strip

$(GZ)/$(BINUTILS_GZ):
	$(WGET) https://mirror.tochlab.net/pub/gnu/binutils/$(BINUTILS_GZ)

Appending to msp430/Makefile


after `make` run `configure` will be run with required options:

`/home/dpon/jupy/msp430/tmp/src/binutils-2.32/configure --disable-nls --target=msp430`

detecting host/target

```
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking target system type... msp430-unknown-none
```

build requires some preinstalled developmen packages:
 * `sudo apt install`
   - `build-essential` `m4` `wget` `make` `gcc g++` `flex` `bison`
   - `xz` `bzip2`
   - `bc` `libncurses-dev` only for Linux kernel build

```
checking for gcc... gcc
checking for g++... g++
checking for bison... bison
checking for m4... m4
checking for flex... flex
```

In [386]:
%%sh
# cd msp430 ; make binutils ; ls -la tmp/src tmp gnu*